# Mercari NLP

In [1]:
import pandas as pd
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/javm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preproccessing

In [2]:
train_df = pd.read_csv("/Users/javm/Desktop/Final-Project/train.tsv", sep = '\t')

In [3]:
train_df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...


In [4]:
# Turn 'brand name' to 0s and 1s 

In [5]:
train_df['brand_name'].isna().value_counts()

False    849853
True     632682
Name: brand_name, dtype: int64

In [25]:
print(train_df['item_description'].dtypes)


object


In [92]:
train_df['category_name'] = train_df['category_name'].str.replace('/', ' ')

In [94]:
train_df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,total_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men Tops T-shirts,NaN,10.0,1,No description yet,MLB Cincinnati Reds T Shirt Size XLMen/Tops/T-...
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics Computers & Tablets Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Razer BlackWidow Chroma KeyboardElectronics/Co...
2,2,AVA-VIV Blouse,1,Women Tops & Blouses Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,AVA-VIV BlouseWomen/Tops & Blouses/Blouseitem_...
3,3,Leather Horse Statues,1,Home Home Décor Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse StatuesHome/Home Décor/Home Déco...
4,4,24K GOLD plated rose,1,Women Jewelry Necklaces,NaN,44.0,0,Complete with certificate of authenticity,24K GOLD plated roseWomen/Jewelry/Necklacesite...
...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women Dresses Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",Free People Inspired DressWomen/Dresses/Mid-Ca...
1482531,1482531,Little mermaid handmade dress,2,Kids Girls 2T-5T Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,Little mermaid handmade dressKids/Girls 2T-5T/...
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors Exercise Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.",21 day fix containers and eating planSports & ...
1482533,1482533,World markets lanterns,3,Home Home Décor Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,World markets lanternsHome/Home Décor/Home Déc...


In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from nltk import WordNetLemmatizer # lemmatizer using WordNet
from nltk.corpus import wordnet # imports WordNet
from nltk import pos_tag # nltk's native part of speech tagging

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/javm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/javm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        pass
    
    def fit(self, data, y = 0):
        
        return self
    
    def transform(self, data, y = 0):
        fully_normalized_corpus = data.apply(self.process_doc)
        
        return fully_normalized_corpus
        
    def process_doc(self, doc):

        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None

        doc_norm = [tok.lower() for tok in word_tokenize(doc) if ((tok.isalpha()) & (tok not in stop_words)) ]

        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)

In [8]:
proc = TextPreprocessor()

In [9]:
train_df['item_description']

0                                         No description yet
1          This keyboard is in great condition and works ...
2          Adorable top with a hint of lace and a key hol...
3          New with tags. Leather horses. Retail for [rm]...
4                  Complete with certificate of authenticity
                                 ...                        
1482530    Lace, says size small but fits medium perfectl...
1482531     Little mermaid handmade dress never worn size 2t
1482532            Used once or twice, still in great shape.
1482533    There is 2 of each one that you see! So 2 red ...
1482534    New with tag, red with sparkle. Firm price, no...
Name: item_description, Length: 1482535, dtype: object

In [32]:
# Dropped less than 10 rows
train_df.dropna(subset = ['item_description'], inplace = True)

In [33]:
train_df['item_description'].isna().value_counts()

False    1482531
Name: item_description, dtype: int64

In [34]:
# Dropped .4%. Not 40%, but .4% as in .004
train_df.dropna(subset = ['category_name'],inplace = True)

In [58]:
train_df['category_name'].isna().value_counts()

False    1476204
Name: category_name, dtype: int64

In [55]:
df1 = proc.fit_transform(train_df['item_description'])

In [95]:
df2 = proc.fit_transform(train_df['category_name'])

In [57]:
df3 = proc.fit_transform(train_df['name'])

In [60]:
df1.shape

(1476204,)

In [86]:
df1

0                                            description yet
1          keyboard great condition work come port test w...
2          adorable top hint lace key hole back pale pink...
3          new tag leather horse retail rm stand foot hig...
4                          complete certificate authenticity
                                 ...                        
1482530    lace say size small fit medium perfectly never...
1482531        little mermaid handmade dress never wear size
1482532                          use twice still great shape
1482533    see so red orange big red orange one world market
1482534         new tag red sparkle firm price free shipping
Name: item_description, Length: 1476204, dtype: object

In [61]:
df2.shape

(1476204,)

In [84]:
df2

0                         
1                     part
2                         
3             décor accent
4                         
                ...       
1482530                   
1482531                   
1482532    sport accessory
1482533       décor accent
1482534                   
Name: category_name, Length: 1476204, dtype: object

In [62]:
df3.shape

(1476204,)

In [85]:
df3

0          mlb cincinnati red t shirt size xl
1            razer blackwidow chroma keyboard
2                                      blouse
3                        leather horse statue
4                              gold plat rise
                          ...                
1482530             free people inspire dress
1482531         little mermaid handmade dress
1482532            day fix container eat plan
1482533                 world market lanterns
1482534                  brand new lux wallet
Name: name, Length: 1476204, dtype: object

In [82]:
df1[0]

'description yet'

''

In [80]:
df12 = df1.str.cat(df2, sep = ' ',)

In [81]:
df12[0]

'description yet '

In [47]:
prc_steps = [('tfid', TfidfVectorizer(min_df = 0.05, max_df = 0.95))]
tfid_pipeline = Pipeline(prc_steps)